Neuroevolution: Exercise 1 
=========
###### Artur Ganzha 10019651
---------	
###### Raul Gorek 10061333
---------	

In [122]:
import numpy as np

In [123]:
X = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
y = np.array([0.0, 1.0, 1.0, 0.0]) 

In [124]:
def xor(a, b):
    return (a or b) and not (a and b)


In [125]:
def relu(x):
    return x * (x > 0)

In [126]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [127]:
# Aufgabe 2
class NeuralNetwork:
    def __init__(self, layers: list):
        self.layers = layers
        for layer in self.layers:
            layer.init()

    def forward_pass(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
class Linear:
    def __init__(self, input_size, output_size, activation = relu):
        self.input_size = input_size
        self.output_size = output_size
        self.activation = activation
    
    def init(self):
        self.W = np.random.uniform(-1, 1,(self.input_size,self.output_size))
    
    def forward(self, x):
        self.in_pass = x
        self.fw = np.dot(self.in_pass, self.W)
        self.act_forward = self.activation(self.fw)
        return self.act_forward




In [128]:
layers = [
    Linear(2,8),
    Linear(8, 4),
    Linear(4,1, sigmoid)
]

In [129]:
neural_net = NeuralNetwork(layers)

In [130]:
batch = np.array([[0,0],[0,1]])
neural_net.forward_pass(batch)

array([[0.5       ],
       [0.17382848]])

In [131]:
# Aufgabe 3
def binary_cross_entropy_loss(prediction, ground_truth):
    return -(ground_truth * np.log(prediction) + (1-ground_truth) * np.log(1-prediction))

In [132]:
def derivative_bcel(prediction, ground_truth):
    x =  np.where(ground_truth == 0, 1.0 / (1.0 - prediction), -1.0 / prediction)
    return x

In [133]:
def derivative_relu(z):
    return z > 0

In [134]:
def derivative_sigmoid(z):
    x = sigmoid(z)
    return x * (1-x)

In [135]:
def backward_pass(self, deriv_loss, learn_rate, prediction, ground_truth):
    deriv = deriv_loss(prediction, ground_truth)
    for layer in reversed(self.layers):
        if layer.activation == relu:
            deriv_act = derivative_relu(layer.fw)
        elif layer.activation == sigmoid:
            deriv_act = derivative_sigmoid(layer.fw)
        print(deriv_act.shape, deriv.shape)
        deriv = deriv * deriv_act
        print(deriv.shape)
        print("--", layer.in_pass.shape)
        update_deriv = np.dot(layer.in_pass.T, deriv)
        print(update_deriv.shape)
        layer.W -= learn_rate * update_deriv




In [136]:
NeuralNetwork.backward_pass = backward_pass

In [137]:
num_epochs = 100
batch = np.array([[0,0],[0,1], [1,0], [1,1]])
labels = np.array([[0.0], [1.0], [1.0], [0.0]]) 
for i in range(num_epochs):
    y_hat = neural_net.forward_pass(batch)
    loss = binary_cross_entropy_loss(y_hat, labels)
    neural_net.backward_pass(derivative_bcel, 0.01, y_hat, labels)
    print(loss)
print(neural_net.forward_pass(batch))

(4, 1) (4, 1)
(4, 1)
-- (4, 4)
(4, 1)
(4, 4) (4, 1)
(4, 4)
-- (4, 8)
(8, 4)
(4, 8) (4, 4)


ValueError: operands could not be broadcast together with shapes (4,4) (4,8) 